<a href="https://colab.research.google.com/github/TetianaHrunyk/NeuralMetaphorsDetection/blob/master/LoadModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface-hub==0.0.8

In [2]:
! pip install  transformers seqeval

     |████████████████████████████████| 2.3MB 9.7MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 901kB 39.2MB/s 
     |████████████████████████████████| 3.3MB 39.0MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=f37ab006a77efb4801474edf1f3f75f25d3982bc4e3f42f22c394efb6227dbd6
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [45]:
model_checkpoint = "distilbert-base-uncased"
batch_size=16
label_list = ["Literal", "Metaphorical"]

In [43]:
import numpy as np

In [4]:
from torch.utils.data import Dataset
import pandas as pd
import torch
import torch.functional as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
from transformers import AutoModelForTokenClassification

In [32]:
from transformers import DataCollatorForTokenClassification, TrainingArguments, Trainer


In [35]:
data_collator = DataCollatorForTokenClassification(tokenizer)
args = TrainingArguments(
    "MetahporsDetection",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [37]:
def predict(sent, trainer, label_all_tokens=False):
    tokenized = tokenizer(sent, is_split_into_words=False, truncation=True)
    tokens = tokenizer.convert_ids_to_tokens(tokenized.input_ids)
    predictions, _, _ = trainer.predict([tokenized])
    labels = np.argmax(predictions[0], axis=1)
    for token, label in zip(tokens[1:-1], labels[1:-1]):
      print("{:15} : {:15}".format(token, label_list[label]))

# Model trained with high ratio of metaphorical data

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-49FpcPUPhJ4GQqs4526upVxf4UBlo4o' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-49FpcPUPhJ4GQqs4526upVxf4UBlo4o" -O fineTunedBert.txt && rm -rf /tmp/cookies.txt

In [38]:
model1 = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=2)
model1.load_state_dict(torch.load('fineTunedBert.txt'))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

<All keys matched successfully>

In [41]:
trainer1 = Trainer(
    model1,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [46]:
predict("Lovely day", trainer1)

lovely          : Metaphorical   
day             : Literal        


# Model trained with mixed data

In [47]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=18_m5XDjt5_tSARDZvKLmhuyr2Scm9Y4L' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=18_m5XDjt5_tSARDZvKLmhuyr2Scm9Y4L" -O fineTunedBert_mixed.txt && rm -rf /tmp/cookies.txt

--2021-06-13 17:52:05--  https://docs.google.com/uc?export=download&confirm=xXlv&id=18_m5XDjt5_tSARDZvKLmhuyr2Scm9Y4L
Resolving docs.google.com (docs.google.com)... 142.250.141.113, 142.250.141.102, 142.250.141.138, ...
Connecting to docs.google.com (docs.google.com)|142.250.141.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-90-docs.googleusercontent.com/docs/securesc/79cfs5bb55fd71dtvcoiqmnia9u7ddc4/vuiked0ni2en2nr0kq01r3mesoa05i5k/1623606675000/00957163558756100188/14158763095630107557Z/18_m5XDjt5_tSARDZvKLmhuyr2Scm9Y4L?e=download [following]
--2021-06-13 17:52:05--  https://doc-14-90-docs.googleusercontent.com/docs/securesc/79cfs5bb55fd71dtvcoiqmnia9u7ddc4/vuiked0ni2en2nr0kq01r3mesoa05i5k/1623606675000/00957163558756100188/14158763095630107557Z/18_m5XDjt5_tSARDZvKLmhuyr2Scm9Y4L?e=download
Resolving doc-14-90-docs.googleusercontent.com (doc-14-90-docs.googleusercontent.com)... 172.217.3.1, 2607:f8b0:4026:800::2001
Connect

In [48]:
model2 = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=2)
model2.load_state_dict(torch.load('fineTunedBert_mixed.txt'))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

<All keys matched successfully>

In [49]:
trainer2 = Trainer(
    model2,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [50]:
predict("Lovely day", trainer2)

lovely          : Literal        
day             : Literal        
